In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

In [26]:
d="""COD,STATE,CITY,AZIM,SET,TEC
ALAAD_0001,AL,MAC,0,1,4
ALAAD_0001,AL,MAC,120,2,4
ALAAD_0001,AL,MAC,120,2,4
ALAAD_0001,AL,MAC,240,3,4
BAPID_0001,BA,SAL,20,1,2
BAPID_0001,BA,SAL,20,1,2
BAPID_0001,BA,SAL,100,2,2
BAPID_0001,BA,SAL,210,3,2
BAPID_0001,BA,SAL,250,3,2
BAPID_0001,BA,SAL,250,3,2
CEMBC_0003,CE,FOR,90,1,4
CEMBC_0003,CE,FOR,80,1,4
CEMBC_0003,CE,FOR,160,2,4
CEMBC_0003,CE,FOR,160,2,4
CEMBC_0003,CE,FOR,170,2,4
CEMBC_0003,CE,FOR,280,3,4"""
df=g(d)
df

,COD,STATE,CITY,AZIM,SET,TEC
0,ALAAD_0001,AL,MAC,0,1,4
1,ALAAD_0001,AL,MAC,120,2,4
2,ALAAD_0001,AL,MAC,120,2,4
3,ALAAD_0001,AL,MAC,240,3,4
4,BAPID_0001,BA,SAL,20,1,2
5,BAPID_0001,BA,SAL,20,1,2
6,BAPID_0001,BA,SAL,100,2,2
7,BAPID_0001,BA,SAL,210,3,2
8,BAPID_0001,BA,SAL,250,3,2
9,BAPID_0001,BA,SAL,250,3,2


In [27]:
df_cut = (
    df.groupby(["COD","TEC","SET"])
    .AZIM
    .agg(lambda x: pd.Series.mode(x).max())
    .reset_index()
)
df_cut

,COD,TEC,SET,AZIM
0,ALAAD_0001,4,1,0
1,ALAAD_0001,4,2,120
2,ALAAD_0001,4,3,240
3,BAPID_0001,2,1,20
4,BAPID_0001,2,2,100
5,BAPID_0001,2,3,250
6,CEMBC_0003,4,1,90
7,CEMBC_0003,4,2,160
8,CEMBC_0003,4,3,280


In [30]:
df_cut.merge(df.drop_duplicates(), on=["COD","TEC","SET", "AZIM"])

,COD,TEC,SET,AZIM,STATE,CITY
0,ALAAD_0001,4,1,0,AL,MAC
1,ALAAD_0001,4,2,120,AL,MAC
2,ALAAD_0001,4,3,240,AL,MAC
3,BAPID_0001,2,1,20,BA,SAL
4,BAPID_0001,2,2,100,BA,SAL
5,BAPID_0001,2,3,250,BA,SAL
6,CEMBC_0003,4,1,90,CE,FOR
7,CEMBC_0003,4,2,160,CE,FOR
8,CEMBC_0003,4,3,280,CE,FOR


In [16]:
df_cut.combine_first(df)

,AZIM,CITY,COD,SET,STATE,TEC
0,0.0,MAC,ALAAD_0001,1.0,AL,4.0
1,120.0,MAC,ALAAD_0001,2.0,AL,4.0
2,240.0,MAC,ALAAD_0001,3.0,AL,4.0
3,20.0,MAC,BAPID_0001,1.0,AL,2.0
4,100.0,SAL,BAPID_0001,2.0,BA,2.0
5,250.0,SAL,BAPID_0001,3.0,BA,2.0
6,90.0,SAL,CEMBC_0003,1.0,BA,4.0
7,160.0,SAL,CEMBC_0003,2.0,BA,4.0
8,280.0,SAL,CEMBC_0003,3.0,BA,4.0
9,250.0,SAL,BAPID_0001,3.0,BA,2.0


In [53]:
idx = [1, 2, 3]
colA = [10, 5, 12]
n = [3, 2, 4]
x1 = [0, 1, 7]
y1 = [4, 0, 4]
x2 = [3, 2, 8]
y2 = [5, 1, 5]
x3 = [4, np.nan, 10]
y3 = [3,np.nan, 3]
x4 = [np.nan, np.nan, 11]
y4 = [np.nan, np.nan, 3]

df = pd.DataFrame(list(zip(idx, colA, n, 
                           x1, y1, x2, y2, x3, y3, x4, y4
                          )), 
                  columns =['idx', 'colA', 'n', 
                            'x1', 'y1', 'x2', 'y2', 
                            'x3', 'y3', 'x4', 'y4'
                           ])
df = df.drop(columns=['n'])
df

,idx,colA,x1,y1,x2,y2,x3,y3,x4,y4
0,1,10,0,4,3,5,4.0,3.0,NaN,NaN
1,2,5,1,0,2,1,NaN,NaN,NaN,NaN
2,3,12,7,4,8,5,10.0,3.0,11.0,3.0


In [57]:
#df.melt(id_vars=(['idx', 'colA'])).dropna()

In [76]:
d="""date_created
2018-12-31 23:30:20+00:00
2018-12-31 23:30:00+00:00
2018-12-31 23:28:09+00:00
2018-12-31 23:28:08+00:00"""
df=pd.read_csv(StringIO(d))
df

,date_created
0,2018-12-31 23:30:20+00:00
1,2018-12-31 23:30:00+00:00
2,2018-12-31 23:28:09+00:00
3,2018-12-31 23:28:08+00:00


In [77]:
df['date_created'] = pd.to_datetime(df.date_created,errors='coerce').dt.tz_localize(None)
df

,date_created
0,2018-12-31 23:30:20
1,2018-12-31 23:30:00
2,2018-12-31 23:28:09
3,2018-12-31 23:28:08


In [78]:
d="""col1,col2,col3,col4,col5,col6,col7
1,test1,t1,Dummy1,result1,10,100.2
2,test1,t1,Dummy2,result2,20,101.2
3,test1,t1,Dummy3,result3,30,102.3
4,test1,t1,Dummy4,result4,40,101.4
5,test2,t1,Dummy1,result1,10,101
6,test2,t1,Dummy2,result2,20,103
7,test2,t1,Dummy3,result3,30,104
8,test2,t1,Dummy4,result4,40,105
9,test3,t1,Dummy1,result1,10,102
10,test3,t1,Dummy2,result2,20,106
11,test3,t1,Dummy3,result3,30,107
12,test3,t1,Dummy5,result4,50,110.2
13,test4,t1,Dummy2,result2,20,120
14,test5,t1,Dummy6,result1,100,88"""
df=g(d)
df

,col1,col2,col3,col4,col5,col6,col7
0,1,test1,t1,Dummy1,result1,10,100.2
1,2,test1,t1,Dummy2,result2,20,101.2
2,3,test1,t1,Dummy3,result3,30,102.3
3,4,test1,t1,Dummy4,result4,40,101.4
4,5,test2,t1,Dummy1,result1,10,101.0
5,6,test2,t1,Dummy2,result2,20,103.0
6,7,test2,t1,Dummy3,result3,30,104.0
7,8,test2,t1,Dummy4,result4,40,105.0
8,9,test3,t1,Dummy1,result1,10,102.0
9,10,test3,t1,Dummy2,result2,20,106.0


In [103]:
d="""A,B,C,D
a,OK,one,col1
b,OK,two,col1
c,OK,two,col2
d,OK,four,NaN
e,OK,five,NaN
f,OK,six,NaN
g,NaN,NaN,col3
h,NaN,NaN,col4
i,NaN,NaN,col5"""
df=g(d)
df

,A,B,C,D
0,a,OK,one,col1
1,b,OK,two,col1
2,c,OK,two,col2
3,d,OK,four,NaN
4,e,OK,five,NaN
5,f,OK,six,NaN
6,g,NaN,NaN,col3
7,h,NaN,NaN,col4
8,i,NaN,NaN,col5


In [104]:
df[['C','D']] = df[['C','D']].fillna('NA') # To keep things simple while dropping col
df.pivot(index='C', columns = 'D', 
         values = 'B').rename_axis(index=None, columns=None).drop(columns='NA', index='NA')

,col1,col2,col3,col4,col5
five,NaN,NaN,NaN,NaN,NaN
four,NaN,NaN,NaN,NaN,NaN
one,OK,NaN,NaN,NaN,NaN
six,NaN,NaN,NaN,NaN,NaN
two,OK,OK,NaN,NaN,NaN


In [ ]:
df.rename_axis()